### Formula 1 Machine Learning Model

** Please be sure to run `pip install fastf1` before running**


In [27]:
import fastf1
import pandas as pd
from IPython.display import display
from datetime import datetime, timezone

now = datetime.now(timezone.utc)

fastf1.Cache.enable_cache('cache')
fastf1.set_log_level('WARNING')

events = fastf1.get_event_schedule(2025, include_testing=True)

# Only include events where both qualifying and race have occurred
required_sessions = ['Session4DateUtc', 'Session5DateUtc']

completed_events = []
for _, event in events.iterrows():
    all_sessions_happened = True
    for session_key in required_sessions:
        session_time = event.get(session_key)

        if pd.isna(session_time):
            all_sessions_happened = False
            break

        if session_time.tzinfo is None:
            session_time = session_time.replace(tzinfo=timezone.utc)

        if session_time >= now:
            all_sessions_happened = False
            break

    if all_sessions_happened:
        completed_events.append(event)

df = pd.DataFrame(completed_events)

# Pull Qualifying and Race Results
combined_results = []

for _, event in df.iterrows():
    try:
        # Load Qualifying
        q_session = fastf1.get_session(2025, event['EventName'], 'Q')
        q_session.load()
        q_results = q_session.results[['Abbreviation', 'Position', 'Q1', 'Q2', 'Q3']].copy()
        q_results.rename(columns={
            'Position': 'QualiPosition',
            'Q1': 'QualiQ1',
            'Q2': 'QualiQ2',
            'Q3': 'QualiQ3'
        }, inplace=True)

        # Load Race
        r_session = fastf1.get_session(2025, event['EventName'], 'R')
        r_session.load()
        r_results = r_session.results[['Abbreviation', 'Position', 'GridPosition', 'Time', 'Status', 'Points']].copy()
        r_results.rename(columns={'Position': 'RacePosition'}, inplace=True)
        r_results.rename(columns={'Time': 'RaceTime'}, inplace=True)

        # Merge
        merged = pd.merge(q_results, r_results, on='Abbreviation', how='inner')
        merged['EventName'] = event['EventName']
        combined_results.append(merged)
    except Exception as e:
        print(f"Failed to pull {event['EventName']} data: {e}")

# Combine events
if combined_results:
    final_df = pd.concat(combined_results, ignore_index=True)
    display(final_df[['EventName', 'Abbreviation', 'QualiPosition', 'QualiQ1', 'QualiQ2', 'QualiQ3',
                      'RacePosition', 'GridPosition', 'RaceTime', 'Status', 'Points']].sort_values(by=['EventName', 'QualiPosition']))
else:
    print("No qualifying and race results could be combined.")

logger      WARNING 	Failed to load schedule from FastF1 backend!
logger      WARNING 	Failed to load schedule from F1 API backend!
logger      WARNING 	Failed to load schedule from Ergast API backend!


ValueError: Failed to load any schedule data.